In [11]:
import numpy as np
import pandas as pd

In [12]:
path = '/home/nivethithan/Downloads/dataset'

In [13]:
train_df=pd.read_csv(path+'/train.csv')

In [14]:
train_df.head()

,image_id,breed
0,a8b3ad1dde,nerodia-erythrogaster
1,8b492b973d,pantherophis-vulpinus
2,929b99ea92,thamnophis-sirtalis
3,bbac7385e2,pantherophis-obsoletus
4,ef776b1488,agkistrodon-contortrix


In [15]:
for i in range(len(train_df['image_id'])):
    train_df['image_id'][i]=train_df['image_id'][i]+'.jpg'

In [16]:
len(train_df['image_id'])

5508

In [17]:
train_df.head()

,image_id,breed
0,a8b3ad1dde.jpg,nerodia-erythrogaster
1,8b492b973d.jpg,pantherophis-vulpinus
2,929b99ea92.jpg,thamnophis-sirtalis
3,bbac7385e2.jpg,pantherophis-obsoletus
4,ef776b1488.jpg,agkistrodon-contortrix


In [18]:
from keras.preprocessing import image
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input,decode_predictions
from keras.layers import Input, Conv2D, MaxPool2D, Dense, Flatten
from keras.models import Model
from keras.utils import to_categorical

In [19]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,)
        # shear_range=0.2,
        # zoom_range=0.2,
        # horizontal_flip=True,
        # width_shift_range=0.1,
        # height_shift_range=0.1

train_generator=train_datagen.flow_from_dataframe(dataframe=train_df,
                                            directory=path + "/train/",
                                            x_col="image_id",
                                            y_col="breed",
                                            subset="training",
                                            batch_size=128,
                                            seed=42,
                                            shuffle=True,
                                            class_mode="categorical",
                                            target_size=(299,299))

Found 5508 validated image filenames belonging to 35 classes.


In [20]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.resnet50 import preprocess_input,decode_predictions
from keras.preprocessing import image

In [21]:
resnet=InceptionResNetV2(include_top=True,weights='imagenet')

225214464/225209952 [==============================] - 43s 0us/step


In [22]:
x=resnet.layers[-2].output
fc1=Dense(35,activation='softmax')(x)

In [23]:
my_model=Model(inputs=resnet.input,outputs=fc1)

In [24]:
from keras.optimizers import Adam

In [25]:
adam=Adam(learning_rate=0.001)

In [26]:
for l in my_model.layers[:-35]:
    #print(l)
    l.trainable = False
my_model.compile(optimizer='adam',loss ="categorical_crossentropy",metrics=["accuracy"])

In [27]:
my_model.fit_generator(train_generator,epochs=3,steps_per_epoch=5508//128)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/3
43/43 [==============================] - 998s 23s/step - loss: 2.6152 - accuracy: 0.2851
Epoch 2/3
43/43 [==============================] - 972s 23s/step - loss: 1.6683 - accuracy: 0.5138
Epoch 3/3
43/43 [==============================] - 969s 23s/step - loss: 1.1401 - accuracy: 0.6742


In [28]:
my_model.save('model.h5')

In [29]:
test_df=pd.read_csv(path + '/test.csv')

In [30]:
for i in range(len(test_df['image_id'])):
    test_df['image_id'][i]=test_df['image_id'][i]+'.jpg'

In [31]:
test_df.head()

,image_id
0,7ede553357.jpg
1,3500b219e4.jpg
2,d43a78d6d9.jpg
3,1f36f26994.jpg
4,8cb85fc58e.jpg


In [32]:
test_df.shape

(2361, 1)

In [33]:
test_datagen = ImageDataGenerator(rescale=1./255,)
        # shear_range=0.2,
        # zoom_range=0.2,
        # horizontal_flip=True,
        # width_shift_range=0.1,
        # height_shift_range=0.1

test_generator=test_datagen.flow_from_dataframe(dataframe=test_df,
                                            directory=path + "/test/",
                                            x_col="image_id",
                                            y_col=None,
                                            batch_size=128,
                                            seed=42,
                                            shuffle=False,
                                            class_mode=None,
                                            target_size=(331,331))

Found 2361 validated image filenames.


In [35]:
y_pred=my_model.predict_generator(test_generator,verbose=1)

19/19 [==============================] - 560s 29s/step


In [36]:
y_pred.shape

(2361, 35)

In [37]:
y_pred_2=np.argmax(y_pred,axis=1)

In [38]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in y_pred_2]

In [39]:
predictions

['agkistrodon-piscivorus',
 'storeria-dekayi',
 'pituophis-catenifer',
 'storeria-dekayi',
 'storeria-dekayi',
 'storeria-dekayi',
 'storeria-dekayi',
 'storeria-dekayi',
 'storeria-dekayi',
 'agkistrodon-piscivorus',
 'pantherophis-alleghaniensis',
 'storeria-dekayi',
 'crotalus-atrox',
 'storeria-dekayi',
 'lampropeltis-californiae',
 'nerodia-sipedon',
 'pantherophis-alleghaniensis',
 'crotalus-atrox',
 'agkistrodon-piscivorus',
 'agkistrodon-piscivorus',
 'storeria-dekayi',
 'agkistrodon-piscivorus',
 'crotalus-horridus',
 'storeria-dekayi',
 'pantherophis-guttatus',
 'crotalus-atrox',
 'storeria-dekayi',
 'pantherophis-guttatus',
 'storeria-dekayi',
 'storeria-dekayi',
 'agkistrodon-piscivorus',
 'storeria-dekayi',
 'thamnophis-sirtalis',
 'storeria-dekayi',
 'thamnophis-sirtalis',
 'storeria-dekayi',
 'lampropeltis-triangulum',
 'storeria-dekayi',
 'pituophis-catenifer',
 'storeria-dekayi',
 'storeria-dekayi',
 'storeria-dekayi',
 'storeria-dekayi',
 'storeria-dekayi',
 'thamnoph

In [40]:
test_df=pd.read_csv(path + '/test.csv')

In [41]:
test_df.head()

,image_id
0,7ede553357
1,3500b219e4
2,d43a78d6d9
3,1f36f26994
4,8cb85fc58e


In [42]:
test_name=test_df['image_id'].values

In [43]:
test_name.shape

(2361,)

In [44]:
data=pd.DataFrame((zip(test_name,predictions)),columns=['image_id','breed'])

In [45]:
data.to_csv('output.csv',index=False)